In [21]:
"""
Movie recommendation system based on genres.
"""

'\nMovie recommendation system based on genres.\n'

In [22]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
# Load movie dataset (Nomainīts, jo dati autoram glabājās lokāli)
movies = pd.read_csv("C:\Bakalaurs_praktiskais\Bakalaura-darbs\movies.csv")
ratings = pd.read_csv(r"C:\Bakalaurs_praktiskais\Bakalaura-darbs\ratings.csv")

In [24]:
# Data preprocessing
movies['genres'] = movies['genres'].apply(lambda x: x.lower().replace('|', ' '))

In [25]:
# Create TF-IDF vectorizer and fit the movie dataset
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(movies['genres'])

In [26]:
#Calculate similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [27]:
def find_movie_index(movie_title, year):
    if year != None:
        movie_title = f"{movie_title} ({year})"
        if movie_title not in movies['title'].values:
            return None
        return movies[movies['title'] == movie_title].index[0]
    else:
        if not any(movies['title'].str.contains(movie_title)):
            return None
        return movies[movies['title'].str.contains(movie_title)].index[0]

In [28]:
def recommend_movies(movie_title, year=None, n_recommendations=5):
    movie_index = find_movie_index(movie_title, year)
    if movie_index is None:
        movie_not_found_message = f"Movie  '{movie_title}' not found in dataset."
        print(movie_not_found_message)
        return None

    sim_scores = list(enumerate(cosine_sim_matrix[movie_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n_recommendations + 1]

    recommended_movie_indices = [i[0] for i in sim_scores]
    return movies.iloc[recommended_movie_indices]

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
import random

def evaluate(movies, ratings, recommend_function, n_recommendations=5, top_n_users=20):
    # Apvieno filmu vērtējumus ar filmām
    data = pd.merge(ratings, movies, on='movieId')
    # Izvēlās tikai aktīvākos lietotājus
    top_users = data['userId'].value_counts().head(top_n_users).index.tolist()
    # Izveido tukšus sarakstus priekš metrikām
    all_precisions, all_recalls, all_f1s = [], [], []
    user_maes, user_rmses = [], []
    
    # Jauni saraksti visiem faktiskajiem un prognozētajiem vērtējumiem, lai aprēķinātu kopējo MAE un RMSE
    all_actual_ratings = []
    all_predicted_ratings = []
    
    # Saraksts, lai saglabātu katra lietotāja metriku rezultātus
    user_metrics = []
    
    for user_id in top_users:
        user_ratings = data[data['userId'] == user_id]
        # Norāda vērtējumu robežu, kuru uzskatīt par pozitīvi novērtētu filmu
        liked_movies = user_ratings[user_ratings['rating'] >= 3.5]
        # Izlaiž lietotājus, kuri ir vērtējuši mazāk par 5 filmām
        if len(liked_movies) < 5:
            continue
        # Uz nejaušību izvēlās vienu filmu, kura ir vērtēta pozitīvi no ietiekumiem
        seed_row = liked_movies.sample(1).iloc[0]
        seed_title = seed_row['title']
        # Apstrādā datus tā, lai nosaukumā netiek iekļauts gads "Title (Year)"
        try:
            if seed_title.strip()[-1] == ')':
                year = int(seed_title.strip()[-5:-1])
                title = seed_title[:-7]  # Noņem gadu no nosaukuma " (YYYY)"
            else:
                year = None
                title = seed_title
        except:
            year = None
            title = seed_title
        # Ģenerē ieteikumus, balstoties uz "seed" filmu
        recs = recommend_function(title, year, n_recommendations=n_recommendations)
        if recs is None:
            continue
        # Ieteikumu saraksts, ko atgriež ieteikumu sistēma
        recommended_titles = set(recs['title'].tolist())
        actual_liked_titles = set(liked_movies['title'].tolist()) - {seed_title}
        # Salīdzina filmas, kuras lietotājs ir novērtējis ar labu vērtējumu pret prognozētajām filmām
        y_true = [1 if title in actual_liked_titles else 0 for title in recommended_titles]
        y_pred = [1] * len(y_true)
        if not y_true:
            continue
        # Aprēķina precizitāti Precision = (sum(y_true)/len(y_pred))
        precision = precision_score(y_true, y_pred, zero_division=0)
        # Aprēķina atsaukumu Recall = (sum(y_true)/len(kopējais filmu skaits, kurš lietotājam ir novērtēts ar pozitīvu vērtējumu))
        recall = recall_score(y_true, y_pred, zero_division=0)
        # Aprēķina F1 = 2*((Precision*Recall)/(Precision+Recall))
        f1 = f1_score(y_true, y_pred, zero_division=0)
        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1s.append(f1)
        
        # MAE & RMSE aprēķins (salīdzina tikai filmas, kurām ir patiesie vērtējumi)
        intersect_titles = recommended_titles & set(user_ratings['title'].tolist())
        if intersect_titles:
            actual_ratings = user_ratings[user_ratings['title'].isin(intersect_titles)]['rating'].values
            predicted_ratings = np.array([4.0] * len(actual_ratings))    # Norāda vērtējumu robežu, kuru uzskatīt par positīvi novērtētu
            
            # Pievieno faktiskos un prognozētos vērtējumus kopējai kolekcijai
            all_actual_ratings.extend(actual_ratings)
            all_predicted_ratings.extend(predicted_ratings)
            
            # Aprēķina MAE un RMSE šim lietotājam
            mae = mean_absolute_error(actual_ratings, predicted_ratings)
            rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
            user_maes.append(mae)
            user_rmses.append(rmse)
            
            # Saglabā lietotāja metrikas
            user_metrics.append({
                'userId': user_id,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'mae': mae,
                'rmse': rmse,
                'rated_count': len(actual_ratings)
            })
    
    # Aprēķina kopējo MAE un RMSE no visiem vērtējumiem
    overall_mae = None
    overall_rmse = None
    if all_actual_ratings:
        overall_mae = mean_absolute_error(all_actual_ratings, all_predicted_ratings)
        overall_rmse = np.sqrt(mean_squared_error(all_actual_ratings, all_predicted_ratings))
    
    # Izprintē rezultātus
    print("\n--- Novērtējums ---")
    print(f"Novērtēto lietotāju skaits: {len(all_precisions)}")
    print(f"Vidējā Precizitāte@{n_recommendations}: {np.mean(all_precisions):.4f}")
    print(f"Videājais Atsaukums@{n_recommendations}: {np.mean(all_recalls):.4f}")
    print(f"Vidējais F1@{n_recommendations}: {np.mean(all_f1s):.4f}")
    
    # Izprintē vidējo lietotāju MAE un RMSE (pa lietotājiem)
    if user_maes and user_rmses:
        print(f"Vidējais lietotāju MAE: {np.mean(user_maes):.4f}")
        print(f"Vidējais lietotāju RMSE: {np.mean(user_rmses):.4f}")
    else:
        print("Lietotāju MAE/RMSE nevar aprēķināt (nav pietiekamu datu).")
    
    # Izprintē kopējo MAE un RMSE (visiem vērtējumiem kopā)
    if overall_mae is not None and overall_rmse is not None:
        print(f"Kopējais MAE: {overall_mae:.4f}")
        print(f"Kopējais RMSE: {overall_rmse:.4f}")
    else:
        print("Kopējo MAE/RMSE nevar aprēķināt (nav pietiekamu datu).")
    
    # Izprintē tabulu ar lietotāju metrikām
    print("\n--- Detalizēta lietotāju statistika ---")
    print(f"{'LietotājaId':<10} {'Precizitāte':>10} {'Atsaukums':>10} {'F1':>10} {'MAE':>10} {'RMSE':>10} {'Atbilstošo filmu skaits ieteikumu sarakstā':>10}")
    print(f"{'-'*70}")
    
    # Sakārto pēc lietotāja ID
    user_metrics.sort(key=lambda x: x['userId'])
    for metrics in user_metrics:
        print(f"{metrics['userId']:<10} {metrics['precision']:>10.4f} {metrics['recall']:>10.4f} {metrics['f1']:>10.4f} {metrics['mae']:>10.4f} {metrics['rmse']:>10.4f} {metrics['rated_count']:>10}")
    
    # Atgriež visas metrikas kā vārdnīcu
    return {
        'precision': np.mean(all_precisions),
        'recall': np.mean(all_recalls),
        'f1': np.mean(all_f1s),
        'lietotāja_mae': np.mean(user_maes) if user_maes else None,
        'lietotāja_rmse': np.mean(user_rmses) if user_rmses else None,
        'kopējais_mae': overall_mae,
        'kopējais_rmse': overall_rmse,
        'lietotāja_metrikas': user_metrics
    }

In [30]:
if __name__ == "__main__":
    year = 1995
    movie_title = "Toy Story"
    n_recommendations = 10
    print(f"Similar recommendations for movie '{movie_title}':")

    recommendations = recommend_movies(movie_title, year, n_recommendations)
    if recommendations is not None:
        print(recommendations)
        
    # Novērtējuma funkcijas izsaukums
    evaluate(movies, ratings, recommend_movies, n_recommendations=20, top_n_users=50)

Similar recommendations for movie 'Toy Story':
      movieId                                              title  \
1706     2294                                        Antz (1998)   
2355     3114                                 Toy Story 2 (1999)   
2809     3754     Adventures of Rocky and Bullwinkle, The (2000)   
3000     4016                   Emperor's New Groove, The (2000)   
3568     4886                              Monsters, Inc. (2001)   
6194    45074                                   Wild, The (2006)   
6486    53121                             Shrek the Third (2007)   
6948    65577                     Tale of Despereaux, The (2008)   
7760    91355  Asterix and the Vikings (Astérix et les Viking...   
8219   103755                                       Turbo (2013)   

                                           genres  
1706  adventure animation children comedy fantasy  
2355  adventure animation children comedy fantasy  
2809  adventure animation children comedy fantas